In [ ]:
pip install --upgrade vis-evaluator

In [ ]:
pip install openai==1.12.0

to make cairosvg work on windows

In [ ]:
import os
os.environ['path'] += r';C:\Program Files\UniConvertor-2.0rc5\dlls'

In [ ]:
import cairosvg

In [ ]:
import openai

In [ ]:
from viseval.agent import Agent, ChartExecutionResult

In [ ]:
# download caht2vis.py: https://github.com/microsoft/VisEval/blob/main/examples/agent/chat2vis.py
# put chat2vis to folder: ...\Python\Lib\site-packages\viseval
from viseval.chat2vis import Chat2vis

In [ ]:
# inside https://github.com/microsoft/VisEval/blob/main/examples/evaluate.py
import argparse
from pathlib import Path

import dotenv
from viseval.chat2vis import Chat2vis

from viseval import Dataset, Evaluator

dotenv.load_dotenv()


def configure_llm(model: str, agent: str):
    if agent == "lida":
        if model in ["gpt-35-turbo", "gpt-4"]:
            from llmx import llm

            return llm(
                provider="openai",
                api_type="azure",
                model=model,
                models={
                    "max_tokens": 4096,
                    "temperature": 0.0,
                },
            )
        else:
            raise ValueError(f"Unknown model {model}")
    else:
        if model == "gemini-pro":
            from langchain_google_genai import ChatGoogleGenerativeAI

            return ChatGoogleGenerativeAI(
                model=model, temperature=0.0, convert_system_message_to_human=True
            )
        elif model in ["gpt-35-turbo", "gpt-4"]:
            from langchain_openai import AzureChatOpenAI

            return AzureChatOpenAI(
                model_name=model,
                max_retries=999,
                temperature=0.0,
                request_timeout=20,
            )
        elif model == "codellama-7b":
            from model.langchain_llama import ChatLlama

            return ChatLlama("../llama_models/CodeLlama-7b-Instruct")
        else:
            raise ValueError(f"Unknown model {model}")


def config_agent(agent: str, model: str, config: dict):
    llm = configure_llm(model, agent)
    if agent == "coml4vis":
        return CoML4VIS(llm, config)
    elif agent == "chat2vis":
        return Chat2vis(llm)
    elif agent == "lida":
        return Lida(llm)
    else:
        raise ValueError(f"Unknown agent {agent}")

